<a href="https://colab.research.google.com/github/griffinsnest/OneHotEncoding/blob/main/oneshotencoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import numpy as np
from sklearn import linear_model as lm
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

university_towns = []

with open('/content/university_towns.txt') as file:
  for line in file:
    if '[edit]' in line:
      state = line
    else:
      university_towns.append((state, line))

university_towns[:10]

towns_df = pd.DataFrame(university_towns, columns=['State', 'RegionName'])

#towns_df['State'] = towns_df['State'].str.replace("\[edit\]\\n", '')

def get_city(item):
  if " (" in item:
    return item[:item.find(')')+1]
  elif '[' in item:
    return item[:item.find('[')]
  else:
    return item

towns_df = towns_df.applymap(get_city)

new = towns_df["RegionName"].str.split(" \(", n = 1, expand = True)
  
# making separate first name column from new data frame
towns_df["City"]= new[0]
  
# making separate last name column from new data frame
towns_df["University"]= new[1]

towns_df["University"]= towns_df["University"].str.replace(')', '')
  
# Dropping old Name columns
towns_df.drop(columns =["RegionName"], inplace = True)

label_encoder = LabelEncoder()

towns_df['StateEncoded'] = label_encoder.fit_transform(towns_df['State'])

towns_df['ScldStateEncoded'] = round(towns_df['StateEncoded'].apply(lambda x: (x - towns_df['StateEncoded'].mean()) / (towns_df['StateEncoded'].std()) ), 2)


#print(towns_df.sample(5))

towns_df = towns_df.head().dropna()

states = towns_df['State']

towns_df = towns_df.drop(['State'], axis=1)

#print(towns_df.head())

data_column_category = towns_df.select_dtypes(exclude=[np.number]).columns
data_column_numeric = towns_df.select_dtypes(include=[np.number]).columns

df_oneshot_getdummies = pd.get_dummies(towns_df[data_column_category], prefix=data_column_category)

#print(df_oneshot_getdummies.sample(5))

final_towns_df = pd.concat([df_oneshot_getdummies, towns_df[data_column_numeric]], axis=1)

final_towns_df = final_towns_df.dropna()

final_towns_df['States'] = states

final_towns_df.head()

X_train, X_test, Y_train, Y_test = train_test_split(final_towns_df[final_towns_df.columns.difference(['StateEncoded', 'ScldStateEncoded', 'States'])], final_towns_df['ScldStateEncoded'], test_size=.25)

#xb = np.array(X_train)
#xb = xb.reshape(-1,1)
#yb = np.array(Y_train)
#yb = yb.reshape(-1,1)

#xb_test = np.array(X_test)
#xb_test = xb.reshape(-1,1)
#yb_test = np.array(Y_test)
#yb_test = yb.reshape(-1,1)

#TestModel = lm.LinearRegression().fit(xb,yb)
#TMscore = TestModel.score(xb_test, yb_test)

#print('\nThe Test model has an linear regression accuracy of:  ', BMscore, sep="")
#print('\nThe parameters for the best model are :\n m = ', bestModel.coef_, '\n b = ', bestModel.intercept_)

#multivariate example
xtm = final_towns_df[final_towns_df.columns.difference(['StateEncoded', 'ScldStateEncoded', 'States'])]
ytm = np.array(final_towns_df['ScldStateEncoded'])
ytm = ytm.reshape(-1,1)

testModelm = lm.LinearRegression().fit(xtm,ytm)
Testscorem = testModelm.score(xtm,ytm)

print('\nUsing multiple input features and avoiding other tax revenue components, gives a linear regression accuracy of:  ', Testscorem, sep="")
print('\nThe parameters for the best multivariate model are :\n m = ', testModelm.coef_, '\n b = ', testModelm.intercept_)


Using multiple input features and avoiding other tax revenue components, gives a linear regression accuracy of:  1.0

The parameters for the best multivariate model are :
 m =  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] 
 b =  [-1.9]
